In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from scipy.io import loadmat
import numpy as np
import os, sys

# Задаём путь к родительской директории вручную
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)


from src.model import FullModel
import src.utils
from src.train import Trainer

from src.dataset import (SurfaceEMGDataset, EMGTransform, extract_emg_labels, make_weighted_sampler, 
                     compute_stats, save_stats, load_stats, GlobalStandardizer, gestures)
from config import (SUBJECTS, WINDOW_SIZE, GLOBAL_SEED, TRAIN_SIZE, BATCH_SIZE, GESTURE_INDEXES, 
                    LEARNING_RATE, EPOCHS, RUN_NAME, VALID_SIZE, EARLY_STOP_THRS)

from sklearn.model_selection import train_test_split

In [2]:
model = FullModel()

In [3]:
os.getcwd()

'/home/etri3060/1_CNN_gestures/bionic_hand_ETRI/notebooks'

In [4]:
checkpoint = torch.load('../logs/model_1/best_model_57.pth')

In [5]:
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [6]:
os.chdir('..')

In [7]:
emg, labels = extract_emg_labels(subjects=SUBJECTS, window_size=WINDOW_SIZE, onehot=False)
emg, labels = gestures(
        emg,
        labels,
        targets=[0,1,2,3],    # или любые твои метки
        relax_shrink=80000,
        rand_seed=GLOBAL_SEED
    )

# NOTE: Разделение данных на обучающую, валидационную и тестовую выборки
emg_train, emg_tmp, labels_train, labels_tmp = train_test_split(emg, labels, train_size=TRAIN_SIZE, random_state=GLOBAL_SEED)
emg_valid, emg_test, labels_valid, labels_test = train_test_split(emg_tmp, labels_tmp, train_size=VALID_SIZE, random_state=GLOBAL_SEED)

100%|██████████| 10/10 [00:02<00:00,  4.30it/s]


In [9]:
means, stds = load_stats('emg_stats.json')    # Импортируем stats и делаем стандартизатор
standardizer = GlobalStandardizer(means, stds)

In [8]:
trainer = Trainer(model, device='cuda', lr=LEARNING_RATE)

In [13]:
test_dataset = SurfaceEMGDataset(emg_test, labels_test, gestures=GESTURE_INDEXES, transform=standardizer)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [16]:
test_loss, test_acc, f1_test = trainer.evaluate(test_dataloader)

print(test_loss)
print(test_acc)
print(f1_test)

0.9807405006304931
0.9094987183138707
0.9094987183138707
